# A/B/n Testing Course - PA James

## 0.1. Imports

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats

from itertools import combinations

from statsmodels.stats.gof import chisquare_effectsize
from statsmodels.stats.power import GofChisquarePower, TTestIndPower
from statsmodels.sandbox.stats.multicomp import multipletests

# 1.0. Load Data

In [19]:
d = { 'variant': [ 'interact', 'connect', 'learn', 'help', 'services' ],
      'visit': [10283, 2742, 2747, 3180, 2064],
      'clicks_all': [3714, 1587, 1652, 1717, 1348],
      'clicks_link': [42, 53, 21, 38, 45] }

data = pd.DataFrame( d )
data['conversion'] = data['clicks_link'] / data['clicks_all']
data

,variant,visit,clicks_all,clicks_link,conversion
0,interact,10283,3714,42,0.011309
1,connect,2742,1587,53,0.033396
2,learn,2747,1652,21,0.012712
3,help,3180,1717,38,0.022132
4,services,2064,1348,45,0.033383


# 2.0. Design de Experimentos

## 2.1. Formulação das Hipóteses

In [20]:
# H0: Não há nenhuma diferença entre o CTR das variantes da página
# H1: Há uma diferença entre os CTR das variantes da página
#
# Teste Bi-Caudal

## 2.2. Parâmetros do Experimentos

In [21]:
k = len( data['clicks_all'] )
actual_dist = data['clicks_link'] / data['clicks_link'].sum()
expected_dist = [ 1 / k ] * k

In [22]:
effect_size = chisquare_effectsize( expected_dist, actual_dist )
alpha = 0.05
power = 0.80

#sample_size = GofChisquarePower().solve_power( 
#    effect_size=effect_size, 
#    alpha =alpha, 
#    power =power,
#    n_bins = ( 2 - 1 ) * ( 5 - 1) )

# T-Test
sample_size = TTestIndPower().solve_power(
    effect_size = effect_size,
    alpha = alpha,
    power = power )

sample_size = np.ceil( sample_size ).astype( int )
print( f'Minimum Sample Size per Variant: {sample_size}' )
print( f'Total Sample Size: {k*sample_size}' )

Minimum Sample Size per Variant: 222
Total Sample Size: 1110


# 3.0. Aplicação do Teste Chi Squared

In [23]:
data['no_clicks_link'] = data['clicks_all'] - data['clicks_link']
df = data[['variant', 'clicks_link', 'no_clicks_link']]
df = df.set_index( 'variant' )
df

,clicks_link,no_clicks_link
variant,,
interact,42,3672
connect,53,1534
learn,21,1631
help,38,1679
services,45,1303


In [24]:
chi2, p_value, dof, ex = stats.chi2_contingency( df )
print( f'Chi Squared:{chi2} - p-value: {p_value}')

Chi Squared:46.33660181942126 - p-value: 2.0959498129984563e-09


# 4.0. Post-hoc Testing

In [28]:
all_combinations = list( combinations( df.index, 2 ) )
p_values = []

for comb in all_combinations:
    new_df = df[( df.index == comb[0] ) | ( df.index == comb[1] )]
    chi2, p_value, dof, ex = stats.chi2_contingency( new_df )
    p_values.append( p_value )
        
## Correction of Benferroni
reject_list, corrected_p_values, _, _ = multipletests( p_values, method='bonferroni' )

In [30]:
for comb, p_val, corr_p_val, reject in zip( all_combinations, p_values, corrected_p_values, reject_list ):
    print( '\n{} p_value:{} corrected_p_values: {} Reject: {}'.format(comb, p_val, corr_p_val, reject))


('interact', 'connect') p_value:5.3676772349808135e-08 corrected_p_values: 5.367677234980813e-07 Reject: True

('interact', 'learn') p_value:0.7616980743361713 corrected_p_values: 1.0 Reject: False

('interact', 'help') p_value:0.0031030587017400212 corrected_p_values: 0.03103058701740021 Reject: True

('interact', 'services') p_value:1.798089447385411e-07 corrected_p_values: 1.7980894473854111e-06 Reject: True

('connect', 'learn') p_value:0.00013292868361715983 corrected_p_values: 0.0013292868361715984 Reject: True

('connect', 'help') p_value:0.06144184057612575 corrected_p_values: 0.6144184057612575 Reject: False

('connect', 'services') p_value:1.0 corrected_p_values: 1.0 Reject: False

('learn', 'help') p_value:0.0508958228881819 corrected_p_values: 0.5089582288818191 Reject: False

('learn', 'services') p_value:0.00020374035733741825 corrected_p_values: 0.0020374035733741825 Reject: True

('help', 'services') p_value:0.07301998638337415 corrected_p_values: 0.7301998638337415 Re

In [ ]:
('interact', 'connect')  -> Há uma relação, diferença, dependência -> 5.367677234980813e-07
('interact', 'services') -> Há uma relação, diferença, dependência -> 1.7980894473854111e-06
('interact', 'help')     -> Há uma relação, diferença, dependência -> 0.03103058701740021 

('connect', 'help')      -> Não há diferença
('connect', 'services')  -> Não há diferença
('help', 'services')     -> Não há diferença